In [1]:
import os, fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import psycopg2
import ast
import jieba
import jieba.analyse
import jieba.posseg as pseg
from pypinyin import pinyin, lazy_pinyin, Style
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
jieba.initialize()

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.577 seconds.
Prefix dict has been built succesfully.


In [2]:
def get_ranking_list():
    connect = psycopg2.connect(dbname = 'xyz',
                           user = 'xyz',
                           password = 'xyz',
                           host = 'analytics.ipm.edu.mo',
                           port = '5432'
                          )
    cur = connect.cursor()
    cur.execute('SELECT ranking.pubdate, bvid, tag, duration, review, is_union_video, play, description, \
                video_review, favorites, title, video_zone, user_space.name, sex, sign, level, birthday, \
                official, vip, live_room \
                FROM xyz.public.ranking, xyz.public.user_space \
                WHERE ranking.mid = user_space.mid \
                AND video_zone IN (71, 32, 208, 122, 17, 29, 130, 37, 209, 137) \
                ORDER BY senddate')
    temp = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
    connect.close()
    
    return temp

In [3]:
ranking_list = get_ranking_list()

In [8]:
ranking_list

,bvid,tag,duration,review,is_union_video,play,description,video_review,favorites,title,video_zone,name,sex,sign,level,birthday,official,vip,live_room
pubdate,,,,,,,,,,,,,,,,,,,
2019-01-01 00:07:42,BV1yt41167Uw,"音乐,日语,花泽香菜,恋爱循环,现场,明星",160,12384,0,4213615,http://beijingzhibo.com/,10055.0,40404.0,花泽香菜《恋爱循环》2018-2019北京卫视跨年晚会电视直播版！,29,彼此之间.,保密,时不时忘记B站密码。,6,05-11,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 2, 'status': 1, 'due_date': 166584960...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2019-01-01 11:04:41,BV1ht411U7wN,"BILIBILI正版,城市猎人 剧场版,城市猎人,シティーハンター,伊仓一惠,玄田哲章,北条...",5407,90,0,54386,圭国共和国总统候选人访日，日本警方为其提供了秘密的保安措施，但他在日本的女儿安娜却成为了敌对...,1350.0,91.0,【剧场版】城市猎人 保镖密令,32,哔哩哔哩番剧,保密,关注最帅的官号，你就成为最帅的粉丝了！↗,6,NaN,"{'role': 3, 'title': '哔哩哔哩番剧区官方账号', 'desc': ''...","{'type': 2, 'status': 1, 'due_date': 196436160...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2019-01-01 11:18:34,BV1Vt411B78T,"综艺,郑云龙,阿云嘎,声入人心,湖南卫视跨年晚会,娱乐",212,2625,0,1606732,http://www.mgtv.com/,13670.0,35648.0,【湖南跨年】声入人心六子：这个劫实在是渡不过去了,71,火星少女Yalulu,女,爱磕CP/综艺上瘾/影视混剪/沉迷b站的高冷逗比 噜噜仙女,5,03-03,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 0, 'status': 0, 'due_date': 0, 'vip_p...","{'roomStatus': 0, 'liveStatus': 0, 'url': '', ..."
2019-01-01 11:31:29,BV1ht41167Um,心理测量者,6822,479,0,138188,VCB-Studio\n世界向禁断的和平（系统）伸出了手。\n2116年——距离常守朱被分配...,2928.0,12061.0,【BD1080P】 PSYCHO-PASS The Movie 2015【天香&amp;A....,32,hungry小浩纸,保密,看完视频点了关注呗。有需要up回复的请私信，进群也请私信。,6,NaN,"{'role': 1, 'title': 'bilibili 知名动漫UP主', 'desc...","{'type': 2, 'status': 1, 'due_date': 163301760...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2019-01-01 20:59:53,BV1ht411B7M5,"音乐选集,乐评盘点,韩国音乐,韩国女团,AOA,EXID,momoland,短裙,上下,韩国...",178,967,0,1237561,大赢家！刚出道就要解散，最终以1首歌曲成功逆袭，韩国最火女团！,1645.0,6450.0,大赢家！刚出道就要解散，最终以1首歌曲成功逆袭韩国最火女团！,130,音乐猛抬头,女,NaN,6,01-01,"{'role': 1, 'title': 'bilibili 知名音乐UP主', 'desc...","{'type': 2, 'status': 1, 'due_date': 161884800...","{'roomStatus': 0, 'liveStatus': 0, 'url': '', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23 15:05:22,BV1WJ411M7FE,"教程,PS,软件,学习,设计,教学视频,平面设计,野生技术协会,Photoshop,PS基础...",33735,194,0,16523,喜欢教程的朋友可以关注投币哦！~ 视频都是现在一点点录制的视频，会慢慢剪辑好之后一一上传，喜...,193.0,1166.0,【Photoshop 教程】3天学完PS 入门到精通到转行【最容易听懂的PS入门基础教程】,209,叫我抠图仔,保密,NaN,4,01-01,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 1, 'status': 0, 'due_date': 157608000...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2019-04-11 13:21:31,BV18b411M7xz,"java,java面试,尚硅谷,java面试题,高频面试题,大厂面试题,java大厂面试",77536,288,0,219723,制作不易，大家记得点个关注，一键三连呀【点赞、投币、收藏】感谢支持~\n本期内容包括\nJU...,10748.0,11096.0,尚硅谷Java大厂面试题第二季(java面试必学，周阳主讲),209,尚硅谷,男,程序员标配，人手一套尚硅谷教程，自学一样拿1万+薪资，官网：atguigu.com/?bz,6,03-18,"{'role': 3, 'title': '尚硅谷官方账号', 'desc': '', 't...","{'type': 2, 'status': 1, 'due_date': 164139840...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2019-07-03 17:33:06,BV1k4411w7sV,前端基础教程,64361,384,0,318314,学完JavaScript基础，我们就可以学习JavaScript核心 DOM 和 BOM操作...,18555.0,10812.0,前端基础进阶-JavaScript核心 DOM BOM操作,208,黑马程序员,保密,教程为传智教育黑马程序员出品，更多教程尽在黑马程序员官网http://www.itheima...,6,01-01,"{'role': 3, 'title': '传智教育旗下官方账号', 'desc': '',...","{'type': 2, 'status': 1, 'due_date': 162273600...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."


In [5]:
current_path= os.getcwd()
ranking_list.to_csv(current_path + '/ranking_list.csv', index=False)

In [6]:
ranking_list = pd.read_csv(current_path + '/ranking_list.csv')

In [7]:
ranking_list['pubdate']=pd.to_datetime(ranking_list['pubdate'])
ranking_list.set_index('pubdate', inplace = True)

# Tag Vectorizer

In [9]:
str(ranking_list['tag'].iloc[0])

'音乐,日语,花泽香菜,恋爱循环,现场,明星'

In [10]:
ranking_list_before = ranking_list.loc['2019-01':'2019-12']
ranking_list_after = ranking_list.loc['2020-01':'2020-12']

row_num_before = ranking_list_before.shape[0]
row_num_after = ranking_list_after.shape[0]

strlist_before = []
strlist_after = []

for i in range(0, row_num_before):
    tag = ranking_list_before['tag'].iloc[i]
    strlist_before.append(str(tag))

for i in range(0, row_num_after):
    tag = ranking_list_after['tag'].iloc[i]
    strlist_after.append(str(tag))

print('完成')

完成


<ipython-input-10-3caed1c471dd>:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  ranking_list_before = ranking_list.loc['2019-01':'2019-12']
<ipython-input-10-3caed1c471dd>:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  ranking_list_after = ranking_list.loc['2020-01':'2020-12']


In [11]:
ranking_list_before.to_csv(current_path + '/ranking_list_before.csv')
ranking_list_after.to_csv(current_path + '/ranking_list_after.csv')

In [12]:
input_before = list(items.replace(',', ' ') for items in strlist_before)
input_after = list(items.replace(',', ' ') for items in strlist_after)

In [13]:
input_before

['音乐 日语 花泽香菜 恋爱循环 现场 明星',
 'BILIBILI正版 城市猎人 剧场版 城市猎人 シティーハンター 伊仓一惠 玄田哲章 北条司 儿玉兼嗣 江上洁 高冈希一 搞笑 热血',
 '综艺 郑云龙 阿云嘎 声入人心 湖南卫视跨年晚会 娱乐',
 '心理测量者',
 '音乐选集 乐评盘点 韩国音乐 韩国女团 AOA EXID momoland 短裙 上下 韩国逆袭女团',
 '中国 纪录片 老字号 原创 文化节目 技艺 工匠精神 商业 传统企业文化 传奇老字号 遇见老字号 卫视 人文 故事 大陆 中华老字号 前世今生 依老塑新 老字号的故事 品牌故事 历史',
 '明星 王一博 程潇 舞蹈 剪辑 跨年',
 '音乐选集 歌曲 音乐 开口跪',
 '热血 机战 神作 战斗 恋爱 花泽香菜 茅野爱衣 校园',
 '综艺 张一山 杨紫 娱乐 高能少年团 真人秀 搞笑视频',
 '原创 财产 监狱 死刑 罪犯 吸烟 喝酒 狱警 化妆品 法律',
 '音乐选集 歌曲 音乐 现场 开口跪 LIVE',
 '娱乐',
 '综艺 火箭少女 蔡徐坤 汪峰 李荣浩 吴谨言 许凯 洛天依 薛之谦 Angelababy杨颖',
 '音乐选集 七子之歌',
 'BILIBILI正版 城市猎人 剧场版 城市猎人 シティーハンター 伊仓一惠 玄田哲章 北条司 儿玉兼嗣 江上洁 高冈希一 搞笑 热血',
 '剧场版 团长 天空被囚者',
 '老番茄 house flipper',
 '敖厂长',
 '搞笑 散人 逍遥散人 橙光 龙雏 后宫',
 '朱广权',
 '经验分享 制作过程 野生技术协会 科技 制作 技术宅 原创 学习 芜湖大司马 大司马',
 '海阔天空 情人 外国网红 美国乐队 黄家驹 夜空中最亮的星 欧美音乐 逃跑计划',
 '为国宝打call 历史 文化 人文 中国',
 '音乐选集 歌曲 音乐',
 '再见2018 实况 单机',
 '搞笑 热血 战斗 梶裕贵 雨宫天 户松遥 杉田智和',
 '薛之谦 现场 达拉崩吧 音乐现场 洛天依',
 '人文 纪实 文化 中国 影像 央视 港珠澳大桥 高铁 飞机 航空',
 '桥本环奈 日本 可爱 美少女',
 '实况解说 中国BOY 单机游戏 独立游戏',
 '爆笑 撒贝宁 主持界一哥',
 '乐评盘点 跨年晚会 车祸现场 

In [14]:
# Tag Vectorizer Before

In [36]:
tfidf_model_before = TfidfVectorizer(min_df=0.01).fit(input_before)

In [37]:
transform_before = tfidf_model_before.transform(input_before)

In [38]:
print(tfidf_model_before.vocabulary_)

{'音乐': 82, '现场': 51, '明星': 42, 'bilibili正版': 2, '搞笑': 33, '综艺': 63, '娱乐': 28, '音乐选集': 84, '乐评盘点': 8, '中国': 7, '纪录片': 58, '原创': 22, '卫视': 20, '人文': 9, '历史': 21, '剪辑': 15, '歌曲': 46, '开口跪': 32, '校园': 44, '真人秀': 52, '搞笑视频': 34, 'live': 6, '经验分享': 61, '制作过程': 14, '野生技术协会': 80, '科技': 54, '制作': 13, '学习': 29, '欧美音乐': 45, '文化': 39, '单机': 17, '音乐现场': 83, '纪实': 57, '央视': 25, '日本': 41, '实况解说': 31, '单机游戏': 18, '独立游戏': 50, '奇幻': 26, '自制': 72, '日常': 40, '翻唱': 66, '周杰伦': 24, '经典': 60, '精选歌单': 56, 'korea相关': 5, 'bgm': 0, 'blackpink': 3, '职场': 70, '美女': 65, '社会': 53, '单机联机': 19, '演唱会': 48, '教学视频': 35, '高考': 86, '华语现场': 16, '吐槽': 23, '脱口秀': 71, '教程': 36, '视频教程': 75, '女神': 27, 'bilibili新星计划': 1, '趣味科普人文': 78, '教育': 37, '编程': 64, '程序员': 55, '设计': 76, '学习心得': 30, '暑期爆肝挑战': 43, '英语': 73, '聊聊我的职业': 69, '演讲': 49, '全能打卡挑战': 10, '辩论': 79, '高中': 85, '求职': 47, '面试': 81, '公开课': 12, '考试': 68, '公务员': 11, '课程': 77, '结构化面试': 62, '考研': 67, '英语学习': 74, 'java': 4, '数学': 38, '线上课堂': 59}


In [39]:
print(tfidf_model_before.get_feature_names())

['bgm', 'bilibili新星计划', 'bilibili正版', 'blackpink', 'java', 'korea相关', 'live', '中国', '乐评盘点', '人文', '全能打卡挑战', '公务员', '公开课', '制作', '制作过程', '剪辑', '华语现场', '单机', '单机游戏', '单机联机', '卫视', '历史', '原创', '吐槽', '周杰伦', '央视', '奇幻', '女神', '娱乐', '学习', '学习心得', '实况解说', '开口跪', '搞笑', '搞笑视频', '教学视频', '教程', '教育', '数学', '文化', '日常', '日本', '明星', '暑期爆肝挑战', '校园', '欧美音乐', '歌曲', '求职', '演唱会', '演讲', '独立游戏', '现场', '真人秀', '社会', '科技', '程序员', '精选歌单', '纪实', '纪录片', '线上课堂', '经典', '经验分享', '结构化面试', '综艺', '编程', '美女', '翻唱', '考研', '考试', '聊聊我的职业', '职场', '脱口秀', '自制', '英语', '英语学习', '视频教程', '设计', '课程', '趣味科普人文', '辩论', '野生技术协会', '面试', '音乐', '音乐现场', '音乐选集', '高中', '高考']


In [40]:
print(transform_before.todense())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.79110168 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [43]:
data_tag_before = pd.DataFrame(transform_before.todense(), columns=tfidf_model_before.get_feature_names(), index=ranking_list_before.bvid)

In [44]:
data_tag_before

,bgm,bilibili新星计划,bilibili正版,blackpink,java,korea相关,live,中国,乐评盘点,人文,...,课程,趣味科普人文,辩论,野生技术协会,面试,音乐,音乐现场,音乐选集,高中,高考
bvid,,,,,,,,,,,,,,,,,,,,,
BV1yt41167Uw,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.535409,0.0,0.000000,0.0,0.0
BV1ht411U7wN,0.0,0.0,0.791102,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
BV1Vt411B78T,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
BV1ht41167Um,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
BV1ht411B7M5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.661376,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.750054,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1nE41117BQ,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
BV1WJ411M7FE,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.362399,0.0,0.000000,0.0,0.000000,0.0,0.0
BV18b411M7xz,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


In [45]:
data_tag_before.to_csv(current_path + '/data_tag_before.csv')

In [46]:
# Tag Vectorizer After

In [47]:
tfidf_model_after = TfidfVectorizer(min_df=0.01).fit(input_after)
transform_after = tfidf_model_after.transform(input_after)

In [48]:
data_tag_after = pd.DataFrame(transform_after.todense(), columns=tfidf_model_after.get_feature_names(), index=ranking_list_after.bvid)

In [49]:
data_tag_after

,bilibili新星计划,bilibili正版,blackpink,diy,hiphop,korea相关,lisa,live,mc,minecraft,...,趣味科普人文,配音,野生技术协会,面试,音乐,音乐现场,高中,高考,高能,黑科技
bvid,,,,,,,,,,,,,,,,,,,,,
BV1JJ411b7gX,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BV1nJ411b7dn,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BV1pJ411b78h,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.72066,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BV1nJ411h7xi,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.475614,0.0,0.0,0.0,0.0,0.0
BV1bJ411b7cB,0.0,0.0,0.0,0.621495,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.607299,0.0,0.494906,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1nv411k7km,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BV1864y1T7Ks,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
BV1Vy4y1z7EX,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [50]:
data_tag_after.to_csv(current_path + '/data_tag_after.csv')

# Title & Description Vectorizer

In [51]:
row_num_before = ranking_list_before['title'].shape[0]
row_num_after = ranking_list_after['title'].shape[0]
strlist_before = []
strlist_after = []

for i in range(0, row_num_before):
    temp = [str(ranking_list_before['title'].iloc[i]) + ' ' + str(ranking_list_before['description'].iloc[i])]
    strlist_before.extend(temp)
for i in range(0, row_num_after):
    temp = [str(ranking_list_after['title'].iloc[i]) + ' ' + str(ranking_list_after['description'].iloc[i])]
    strlist_after.extend(temp)
    
print('完成')

完成


In [52]:
strlist_before

['花泽香菜《恋爱循环》2018-2019北京卫视跨年晚会电视直播版！ http://beijingzhibo.com/',
 '【剧场版】城市猎人 保镖密令 圭国共和国总统候选人访日，日本警方为其提供了秘密的保安措施，但他在日本的女儿安娜却成为了敌对势力的袭击目标。为了保护安娜的人身安全，冴羽獠受野上探长的委托成为她的贴身保镖。面对年轻美貌的安娜探员，好色的冴羽獠将如何完成他的任务呢？ ',
 '【湖南跨年】声入人心六子：这个劫实在是渡不过去了 http://www.mgtv.com/',
 '【BD1080P】 PSYCHO-PASS The Movie 2015【天香&amp;A.I.R.nesSub】 VCB-Studio\n世界向禁断的和平（系统）伸出了手。\n2116年——距离常守朱被分配到厚生省公安局刑事课，已经过了大约4年。\n日本政府终于开始向全世界输出西比拉系统以及多隆。长期处于内战状态下的SEAUn（东南亚联盟）的汉议长，在首都香巴拉弗洛特采用了西比拉系统。曾经是枪弹纷飞的纷争地带中心的水上都市香巴拉弗洛特，在一瞬间就成功得到了和平。西比拉系统的实验取得了漂亮的成果——看起来是这样。\n此时，日本遭到全副武装的偷渡者入侵。他们深晓日本的警备体制，躲过西比拉系统的监视开始实施恐怖行为。为了',
 '大赢家！刚出道就要解散，最终以1首歌曲成功逆袭韩国最火女团！ 大赢家！刚出道就要解散，最终以1首歌曲成功逆袭，韩国最火女团！',
 '【纪录片】《传奇老字号》第一集《平遥牛肉》 《传奇老字号》是一档赋予老字号新形象、激活老传承新力量的原创文化节目。节目聚焦华夏先人技艺与智慧的传承，聚焦精益求精的工匠精神和诚信为本的商业道德。它以全国各老字号企业为基点，用最新的电影化节目形式，配以多名专家采访，探寻百年商业秘诀，弘扬传统企业文化，扩大老字号影响力，让更多的年轻人认识老字号，了解老字号，喜爱老字号。',
 '【王一博X程潇】跨年再合作《Fire》 捏下巴超撩 -',
 '12岁小女孩烟嗓翻唱《岁月神偷》火了，被天使吻过的嗓子 最近有一首歌曲《岁月神偷》是非常的火， 有很多的人都去翻唱这首歌曲，而要说道最惊艳的莫过于一位12岁的小女孩了，她的嗓音和她的年龄让很多人都表示赞叹',
 '【BD1080P+】Infinite Stratos/无限斯特拉托斯 第一季

In [53]:
# Title & Description Before

In [54]:
sent_words_before = list(jieba.cut(items) for items in strlist_before)
input_before = [' '.join(items) for items in sent_words_before]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.589 seconds.
Prefix dict has been built succesfully.


In [55]:
input_before

['花泽 香菜 《 恋爱 循环 》 2018 - 2019 北京 卫视 跨年 晚会 电视直播 版 ！   http : / / beijingzhibo . com /',
 '【 剧场版 】 城市猎人   保镖 密令   圭国 共和国 总统 候选人 访日 ， 日本 警方 为 其 提供 了 秘密 的 保安 措施 ， 但 他 在 日本 的 女儿 安娜 却 成为 了 敌对势力 的 袭击 目标 。 为了 保护 安娜 的 人身安全 ， 冴羽 獠受 野上 探长 的 委托 成为 她 的 贴身 保镖 。 面对 年轻美貌 的 安娜 探员 ， 好色 的 冴羽 獠 将 如何 完成 他 的 任务 呢 ？  ',
 '【 湖南 跨年 】 声入 人心 六子 ： 这个 劫 实在 是 渡 不 过去 了   http : / / www . mgtv . com /',
 '【 BD1080P 】   PSYCHO - PASS   The   Movie   2015 【 天香 & amp ; A . I . R . nesSub 】   VCB - Studio \n 世界 向 禁断 的 和平 （ 系统 ） 伸出 了 手 。 \n 2116 年 — — 距离 常守 朱 被 分配 到 厚生省 公安局 刑事 课 ， 已经 过 了 大约 4 年 。 \n 日本政府 终于 开始 向 全世界 输出 西 比拉 系统 以及 多隆 。 长期 处于 内战 状态 下 的 SEAUn （ 东南亚 联盟 ） 的 汉 议长 ， 在 首都 香巴拉 弗洛 特 采用 了 西 比拉 系统 。 曾经 是 枪弹 纷飞 的 纷争 地带 中心 的 水上 都市 香巴拉 弗洛 特 ， 在 一瞬间 就 成功 得到 了 和平 。 西 比拉 系统 的 实验 取得 了 漂亮 的 成果 — — 看起来 是 这样 。 \n 此时 ， 日本 遭到 全副武装 的 偷渡者 入侵 。 他们 深晓 日本 的 警备 体制 ， 躲过 西 比拉 系统 的 监视 开始 实施 恐怖 行为 。 为了',
 '大赢家 ！ 刚出道 就要 解散 ， 最终 以 1 首歌曲 成功 逆袭 韩国 最火 女团 ！   大赢家 ！ 刚出道 就要 解散 ， 最终 以 1 首歌曲 成功 逆袭 ， 韩国 最火 女团 ！',
 '【 纪录片 】 《 传奇 老字号 》 第一集 《 平遥 牛肉 》  

In [56]:
tfidf_model_before = TfidfVectorizer(min_df=0.01).fit(input_before)
transform_before = tfidf_model_before.transform(input_before)

In [57]:
data_desc_before = pd.DataFrame(transform_before.todense(), columns=tfidf_model_before.get_feature_names(), index=ranking_list_before.bvid)

In [58]:
data_desc_before

,00,10,100,1080p,11,12,20,2019,2020,30,...,问题,需要,非常,面试,韩国,音乐,频道,高清,高考,高能
bvid,,,,,,,,,,,,,,,,,,,,,
BV1yt41167Uw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481610,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV1ht411U7wN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV1Vt411B78T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV1ht41167Um,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV1ht411B7M5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.681051,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1nE41117BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.189605,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV1WJ411M7FE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.14275,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
BV18b411M7xz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133271,0.0,0.0,...,0.00000,0.000000,0.0,0.268487,0.000000,0.0,0.0,0.0,0.0,0.0


In [59]:
data_desc_before.to_csv(current_path + '/data_desc_before.csv')

In [60]:
# Title & Description After

In [61]:
sent_words_after = list(jieba.cut(items) for items in strlist_after)
input_after = [' '.join(items) for items in sent_words_after]

In [62]:
input_after

['腾格尔   《 野狼 disco 》   芒果 台',
 '【 萨 顶顶 】 《 左手指 月 》 2020 北京 卫视 环球 跨年 冰雪 盛典 超 清版   beijingzhibo . com',
 '【 潘玮柏 & amp ; 冯提 莫 】 《 不得不 爱 》 提莫 甜美 开唱 ， 2019 - 2020 江苏 卫视 跨年 演唱会 cut   weibo',
 '兄弟姐妹 们 ！ 这 到底 咋 劝 啊 ！ 啊 ？   nan',
 '2019 弹珠 比赛 10 大高光 时刻   转载 ： https : / / www . youtube . com / watch ? v = VQebp - AA4PY \n 感谢 ： Jelle &# 39 ; s   Marble   Runs 的 分享 \n More   marble   news :   https : / / www . marble - sports . com / hub \n Discord :   https : / / discord . gg / marbles',
 '2020 江苏 卫视 跨年 晚会 ， 王力宏 、 谭 维维 再现 《 缘分 一道 桥 》 超强 现场 ， 完美 诠释 中国 风   https : / / v . youku . com / v _ show / id _ XNDQ5MDIwMjA2MA = = . html ? spm = a2h0k.11417342 . soresults . dtitle \n 江苏 卫视 2019 - 2020 跨年 演唱会 ： 王力宏   谭 维维 《 缘分 一道 桥 》 ， 完美 诠释 中国 风',
 '江苏 卫视 2020 跨年 演唱会   宝石 GEM 、 陈伟霆 《 野狼 disco 》 蹦迪 现场   江苏 卫视 跨年 演唱会 \n 江苏 卫视 2020 跨年 演唱会   宝石 GEM 、 陈伟霆 《 野狼 disco 》 蹦迪 现场',
 '【 纪录片 】 但是 还有 书籍   第 4 集   设计师 的 纸 上 王国   思想 无形 ， 幸好 有 了 语言 ， 有 了 文字 ； 记忆 是 不 可靠 的 东西 ， 幸好 有 了 书籍 ； 时间 太快 ， 人们 太忙 ， 选择 太 多 ， 幸好 有 了 书籍 设计

In [63]:
tfidf_model_after = TfidfVectorizer(min_df=0.01).fit(input_after)
transform_after = tfidf_model_after.transform(input_after)

In [64]:
data_desc_after = pd.DataFrame(transform_after.todense(), columns=tfidf_model_after.get_feature_names(), index=ranking_list_after.bvid)

In [65]:
data_desc_after

,01,02,03,10,11,12,20,2019,2020,39,...,面前,面对,面试,韩国,音乐,频道,高中,高中生,高清,高考
bvid,,,,,,,,,,,,,,,,,,,,,
BV1JJ411b7gX,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1nJ411b7dn,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.780578,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1pJ411b78h,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.576429,0.440421,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1nJ411h7xi,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1bJ411b7cB,0.0,0.0,0.0,0.234001,0.0,0.0,0.0,0.287961,0.000000,0.28639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1nv411k7km,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1864y1T7Ks,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Vy4y1z7EX,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
data_desc_after.to_csv(current_path + '/data_desc_after.csv')

# Comment Vectorizer

In [1]:
def get_comment_list(zone):    
    path = '/mnt/sdb1/Downloads/project/preprocess/comment'
    result = ['{}.csv'.format(zone)]
    return result

In [3]:
# 获取分区列表
zonelist = ['71', '32', '208', '122', '17', '29', '130', '37', '209', '137']
# 获取分区数量
zone_num = len(zonelist)
strlist_before = []
strlist_after = []

for z in range(0, zone_num):
    zone_file_list = get_comment_list(zonelist[z])
    
    comment = pd.read_csv('/mnt/sdb1/Downloads/project/preprocess/comment/{}'.format(zone_file_list[0]), engine='python')
    comment['ctime'] = pd.to_datetime(comment['ctime'], unit = 's')
    comment.set_index('ctime', inplace = True)
    
    comment_before = comment.loc['2019-01':'2019-12']
    comment_after = comment.loc['2020-01':'2020-12']
    
    row_num_before = comment_before.shape[0]
    row_num_after = comment_after.shape[0]
        
    for b in range(0, row_num_before):
        # Before
        content = comment_before['content'].iloc[b]
        temp = ast.literal_eval(content)
        text = temp['message']
        strlist_before.append(text)
    for a in range(0, row_num_after):
        # After
        content = comment_after['content'].iloc[a]
        temp = ast.literal_eval(content)
        text = temp['message']
        strlist_after.append(text)
        
    del comment, comment_before, comment_after
    print(zone_file_list[0], ': 处理完成')
    
print('完成')

<ipython-input-3-781236d2a8ed>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-3-781236d2a8ed>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


71.csv : 处理完成


<ipython-input-3-781236d2a8ed>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-3-781236d2a8ed>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


32.csv : 处理完成


<ipython-input-3-781236d2a8ed>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-3-781236d2a8ed>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


...

37.csv : 处理完成


<ipython-input-3-781236d2a8ed>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-3-781236d2a8ed>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


209.csv : 处理完成


<ipython-input-3-781236d2a8ed>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-3-781236d2a8ed>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


137.csv : 处理完成
完成


In [9]:
len(index_before)

623670

In [6]:
# 获取分区列表
zonelist = ['71', '32', '208', '122', '17', '29', '130', '37', '209', '137']
# 获取分区数量
zone_num = len(zonelist)
index_before = []
index_after = []

for z in range(0, zone_num):
    zone_file_list = get_comment_list(zonelist[z])
    
    comment = pd.read_csv('/mnt/sdb1/Downloads/project/preprocess/comment/{}'.format(zone_file_list[0]), engine='python')
    comment['ctime'] = pd.to_datetime(comment['ctime'], unit = 's')
    comment.set_index('ctime', inplace = True)
    
    comment_before = comment.loc['2019-01':'2019-12']
    comment_after = comment.loc['2020-01':'2020-12']
    
    row_num_before = comment_before.shape[0]
    row_num_after = comment_after.shape[0]
        
    for b in range(0, row_num_before):
        # Before
        content = comment_before['bvid'].iloc[b]
        index_before.append(content)
    for a in range(0, row_num_after):
        # After
        content = comment_after['bvid'].iloc[a]
        index_after.append(content)
        
    del comment, comment_before, comment_after
    print(zone_file_list[0], ': 处理完成')
    
print('完成')

<ipython-input-6-6caee12b6eb4>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-6-6caee12b6eb4>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


71.csv : 处理完成


<ipython-input-6-6caee12b6eb4>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-6-6caee12b6eb4>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


32.csv : 处理完成


<ipython-input-6-6caee12b6eb4>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-6-6caee12b6eb4>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


...

37.csv : 处理完成


<ipython-input-6-6caee12b6eb4>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-6-6caee12b6eb4>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


209.csv : 处理完成


<ipython-input-6-6caee12b6eb4>:15: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_before = comment.loc['2019-01':'2019-12']
<ipython-input-6-6caee12b6eb4>:16: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  comment_after = comment.loc['2020-01':'2020-12']


137.csv : 处理完成
完成


In [10]:
# Comment Before

In [11]:
sent_words_before = list(jieba.cut(items) for items in strlist_before)
input_before = [' '.join(items) for items in sent_words_before]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.588 seconds.
Prefix dict has been built succesfully.


In [12]:
tfidf_model_before = TfidfVectorizer(min_df=0.01).fit(input_before)
transform_before = tfidf_model_before.transform(input_before)

In [13]:
data_comment_before = pd.DataFrame(transform_before.todense(), columns=tfidf_model_before.get_feature_names(), index=index_before)

In [14]:
data_comment_before

,doge,up,一下,一个,一些,一定,一样,一点,一直,一起,...,这么,这些,这是,这样,这种,那个,那么,那些,问题,需要
BV1Wt41147UV,0.0,0.0,0.000000,0.0,0.0,0.262352,0.000000,0.0,0.238199,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Wt41147UV,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Wt41147UV,0.0,0.0,0.120143,0.0,0.0,0.000000,0.124497,0.0,0.000000,0.139071,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Wt41147UV,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Wt41147UV,0.0,0.0,0.000000,0.0,0.0,0.000000,0.230784,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1AJ411Y7FM,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1AJ411Y7FM,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1AJ411Y7FM,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1AJ411Y7FM,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
current_path= os.getcwd()
data_comment_before.to_csv(current_path + '/data_comment_before.csv')

In [17]:
# Comment After

In [18]:
sent_words_after = list(jieba.cut(items) for items in strlist_after)
input_after = [' '.join(items) for items in sent_words_after]

In [19]:
tfidf_model_after = TfidfVectorizer(min_df=0.01).fit(input_after)
transform_after = tfidf_model_after.transform(input_after)

In [20]:
data_comment_after = pd.DataFrame(transform_after.todense(), columns=tfidf_model_after.get_feature_names(), index=index_after)

In [21]:
data_comment_after

,call,doge,ok,tv,up,一下,一个,一些,一定,一样,...,这么,这些,这是,这样,这种,那个,那么,那些,问题,需要
BV1Wt41147UV,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Dt411W7aL,0.0,0.0,0.0,0.0,0.0,0.0,0.45495,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Dt411W7aL,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.500529,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Dt411W7aL,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.382842,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Dt411W7aL,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1Ba411w7Ko,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Ba411w7Ko,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Ba411w7Ko,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1Ba411w7Ko,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
data_comment_after.to_csv(current_path + '/data_comment_after.csv')

# Live Comment Vectorizer

In [1]:
import os, fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import psycopg2
import ast
import psycopg2
import jieba
import jieba.analyse
import jieba.posseg as pseg
from pypinyin import pinyin, lazy_pinyin, Style
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
jieba.initialize()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.440 seconds.
Prefix dict has been built succesfully.


In [2]:
def get_file_list(zone):    
    path = '/mnt/sdb1/Downloads/project/preprocess/danmu'
    result = []
    for root, dirs, files in os.walk(path):
        for file in files: 
            if file.startswith('{}-'.format(zone)):
                result.append(os.path.join(file))
                
    result.append('{}.csv'.format(zone))
    return result

In [3]:
# 获取分区列表
zonelist = ['71', '32', '208', '122', '17', '29', '130', '37', '209', '137']
# 获取分区数量
zone_num = len(zonelist)
strlist_before = []
index_before = []

for i in range(0, zone_num):
    zone_file_list = get_file_list(zonelist[i])
    zone_file_num = len(zone_file_list)
    
    for f in range(0, 2):
        temp = pd.read_csv('/mnt/sdb1/Downloads/project/preprocess/danmu/{}'.format(zone_file_list[f*10]), \
                           engine='python')
        temp['dm_timestamp'] = pd.to_datetime(temp['dm_timestamp'], unit = 's')
        temp.set_index('dm_timestamp', inplace = True)

        danmu_before = temp.loc['2019-01':'2019-12']
        row_num_before = danmu_before.shape[0]

        for b in range(0, row_num_before):
            # Before
            content = danmu_before['dm_text'].iloc[b]
            strlist_before.append(content)
            index = danmu_before['bvid'].iloc[b]
            index_before.append(index)
        
        del temp, danmu_before
        print(zone_file_list[f*10], ': 处理完成')
        
print('所有分区处理完成')

<ipython-input-3-020b91eed629>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_before = temp.loc['2019-01':'2019-12']


71-1.csv : 处理完成


<ipython-input-3-020b91eed629>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_before = temp.loc['2019-01':'2019-12']


71-11.csv : 处理完成


<ipython-input-3-020b91eed629>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_before = temp.loc['2019-01':'2019-12']


...

209-11.csv : 处理完成


<ipython-input-3-020b91eed629>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_before = temp.loc['2019-01':'2019-12']


137-1.csv : 处理完成


<ipython-input-3-020b91eed629>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_before = temp.loc['2019-01':'2019-12']


137-11.csv : 处理完成
所有分区处理完成


In [ ]:
# Live Comment Before

In [4]:
sent_words_before = list(jieba.cut(str(items)) for items in strlist_before)
input_before = [' '.join(items) for items in sent_words_before]
del strlist_before, sent_words_before

In [8]:
tfidf_model_before = TfidfVectorizer(min_df=0.002).fit(input_before)
transform_before = tfidf_model_before.transform(input_before)
del input_before

In [9]:
temp = pd.DataFrame(transform_before.todense(), columns=tfidf_model_before.get_feature_names(), index=index_before)
temp.index.name='bvid'
del transform_before, index_before

In [10]:
temp

,10,2333,23333,233333,2333333,84,bgm,henry,pi,一下,...,这首,那个,那么,金宇,金宇硕,金曜汉,阿比,韩哥,首歌,高能
bvid,,,,,,,,,,,,,,,,,,,,,
BV114411q7ji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411x7NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411x7NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411x7NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411x7NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
header = temp.columns.values
header_num = len(header)
new_header = []
str = ' '

for i in range(0, header_num):
    text = str.join(lazy_pinyin(header[i]))
    new_header.append(text)

In [12]:
current_path= os.getcwd()
temp.to_csv(current_path + '/data_danmu_before.csv', header=new_header)
del temp

In [ ]:
# Live Comment After

In [3]:
# 获取分区列表
zonelist = ['71', '32', '208', '122', '17', '29', '130', '37', '209', '137']
# 获取分区数量
zone_num = len(zonelist)
strlist_after = []
index_after = []

for i in range(0, zone_num):
    zone_file_list = get_file_list(zonelist[i])
    zone_file_num = len(zone_file_list)
    
    for f in range(0, 2):
        temp = pd.read_csv('/mnt/sdb1/Downloads/project/preprocess/danmu/{}'.format(zone_file_list[f*10]), \
                           engine='python')
        temp['dm_timestamp'] = pd.to_datetime(temp['dm_timestamp'], unit = 's')
        temp.set_index('dm_timestamp', inplace = True)

        danmu_after = temp.loc['2020-01':'2020-12']
        row_num_after = danmu_after.shape[0]

        for a in range(0, row_num_after):
            # After
            content = danmu_after['dm_text'].iloc[a]
            strlist_after.append(content)
            index = danmu_after['bvid'].iloc[a]
            index_after.append(index)
        
        del temp, danmu_after
        print(zone_file_list[f*10], ': 处理完成')
        
print('所有分区处理完成')

<ipython-input-3-254c43870c57>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_after = temp.loc['2020-01':'2020-12']


71-1.csv : 处理完成


<ipython-input-3-254c43870c57>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_after = temp.loc['2020-01':'2020-12']


71-11.csv : 处理完成


<ipython-input-3-254c43870c57>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_after = temp.loc['2020-01':'2020-12']


...

209-11.csv : 处理完成


<ipython-input-3-254c43870c57>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_after = temp.loc['2020-01':'2020-12']


137-1.csv : 处理完成


<ipython-input-3-254c43870c57>:18: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  danmu_after = temp.loc['2020-01':'2020-12']


137-11.csv : 处理完成
所有分区处理完成


In [4]:
sent_words_after = list(jieba.cut(str(items)) for items in strlist_after)
input_after = [' '.join(items) for items in sent_words_after]
del strlist_after, sent_words_after

In [5]:
tfidf_model_after = TfidfVectorizer(min_df=0.002).fit(input_after)
transform_after = tfidf_model_after.transform(input_after)
del input_after

In [6]:
temp = pd.DataFrame(transform_after.todense(), columns=tfidf_model_after.get_feature_names(), index=index_after)
temp.index.name='bvid'
del transform_after, index_after

In [7]:
temp

,01,04,10,12,13,17,18,19,20,2020,...,这个,这么,这是,这样,这里,这首,那个,那么,问题,高能
bvid,,,,,,,,,,,,,,,,,,,,,
BV114411M7q7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411M7q7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411M7q7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411M7q7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV114411M7q7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BV1st411s7wu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
header = temp.columns.values
header_num = len(header)
new_header = []
str = ' '

for i in range(0, header_num):
    text = str.join(lazy_pinyin(header[i]))
    new_header.append(text)

In [10]:
current_path= os.getcwd()
temp.to_csv(current_path + '/data_danmu_after.csv', header=new_header)
del temp

# Ranking List Preprocess

In [ ]:
# Ranking List Before

In [2]:
current_path= os.getcwd()
ranking_list_before = pd.read_csv(current_path + '/ranking_list_before.csv', index_col='pubdate')

In [8]:
ranking_list_before.head(10)

,bvid,tag,duration,review,is_union_video,play,description,video_review,favorites,title,video_zone,name,sex,sign,level,birthday,official,vip,live_room
pubdate,,,,,,,,,,,,,,,,,,,
2019-01-01 00:07:42,BV1yt41167Uw,"音乐,日语,花泽香菜,恋爱循环,现场,明星",160,12384,0,4213615,http://beijingzhibo.com/,10055.0,40404.0,花泽香菜《恋爱循环》2018-2019北京卫视跨年晚会电视直播版！,29,彼此之间.,2,时不时忘记B站密码。,6,05-11,0,2,1
2019-01-01 11:04:41,BV1ht411U7wN,"BILIBILI正版,城市猎人 剧场版,城市猎人,シティーハンター,伊仓一惠,玄田哲章,北条...",5407,90,0,54386,圭国共和国总统候选人访日，日本警方为其提供了秘密的保安措施，但他在日本的女儿安娜却成为了敌对...,1350.0,91.0,【剧场版】城市猎人 保镖密令,32,哔哩哔哩番剧,2,关注最帅的官号，你就成为最帅的粉丝了！↗,6,NaN,3,2,1
2019-01-01 11:18:34,BV1Vt411B78T,"综艺,郑云龙,阿云嘎,声入人心,湖南卫视跨年晚会,娱乐",212,2625,0,1606732,http://www.mgtv.com/,13670.0,35648.0,【湖南跨年】声入人心六子：这个劫实在是渡不过去了,71,火星少女Yalulu,1,爱磕CP/综艺上瘾/影视混剪/沉迷b站的高冷逗比 噜噜仙女,5,03-03,0,0,0
2019-01-01 11:31:29,BV1ht41167Um,心理测量者,6822,479,0,138188,VCB-Studio\n世界向禁断的和平（系统）伸出了手。\n2116年——距离常守朱被分配...,2928.0,12061.0,【BD1080P】 PSYCHO-PASS The Movie 2015【天香&amp;A....,32,hungry小浩纸,2,看完视频点了关注呗。有需要up回复的请私信，进群也请私信。,6,NaN,1,2,1
2019-01-01 20:59:53,BV1ht411B7M5,"音乐选集,乐评盘点,韩国音乐,韩国女团,AOA,EXID,momoland,短裙,上下,韩国...",178,967,0,1237561,大赢家！刚出道就要解散，最终以1首歌曲成功逆袭，韩国最火女团！,1645.0,6450.0,大赢家！刚出道就要解散，最终以1首歌曲成功逆袭韩国最火女团！,130,音乐猛抬头,1,NaN,6,01-01,1,2,0
2019-01-01 21:15:00,BV14t411B7wx,"中国,纪录片,老字号,原创,文化节目,技艺,工匠精神,商业,传统企业文化,传奇老字号,遇见老...",2602,363,0,217988,《传奇老字号》是一档赋予老字号新形象、激活老传承新力量的原创文化节目。节目聚焦华夏先人技艺与...,1587.0,216.0,【纪录片】《传奇老字号》第一集《平遥牛肉》,37,哔哩哔哩纪录片,0,莫得感情偶尔咕咕的纪录菌,6,01-01,3,2,1
2019-01-01 15:55:08,BV1Ht411B7kW,"明星,王一博,程潇,舞蹈,剪辑,跨年",182,951,0,1324521,-,2625.0,15840.0,【王一博X程潇】跨年再合作《Fire》 捏下巴超撩,137,娱乐小主宰,2,努力努力up,5,10-19,0,0,0
2019-01-02 00:25:06,BV11t411B7ze,"音乐选集,歌曲,音乐,开口跪",125,501,0,1913856,最近有一首歌曲《岁月神偷》是非常的火， 有很多的人都去翻唱这首歌曲，而要说道最惊艳的莫过于一...,1225.0,9100.0,12岁小女孩烟嗓翻唱《岁月神偷》火了，被天使吻过的嗓子,130,夜店嗨歌,2,盘点最好听的音乐，让我们在音乐中相识,5,NaN,0,1,1
2019-01-02 12:10:52,BV1it411B7W3,"热血,机战,神作,战斗,恋爱,花泽香菜,茅野爱衣,校园",19038,410,0,475386,轻之国度字幕组制作\n主人公织斑一夏在高中的入学考试时误入IS操纵者育成学校“IS学园”的考...,21521.0,17242.0,【BD1080P+】Infinite Stratos/无限斯特拉托斯 第一季合集【轻之国度】,32,最想念der季节,0,UP苦流量久矣,6,11-13,1,2,1


In [4]:
ranking_list_before['sex'] = ranking_list_before['sex'].map({'男': 0, '女': 1, '保密': 2})

In [ ]:
# Official

In [5]:
row_num = ranking_list_before.shape[0]

for i in range(0, row_num):
    content = ranking_list_before['official'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['role']
    ranking_list_before['official'].iloc[i] = text

/home/xyk/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Vip

In [6]:
row_num = ranking_list_before.shape[0]

for i in range(0, row_num):
    content = ranking_list_before['vip'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['type']
    ranking_list_before['vip'].iloc[i] = text

In [ ]:
# Live Room

In [7]:
row_num = ranking_list_before.shape[0]

for i in range(0, row_num):
    content = ranking_list_before['live_room'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['roomStatus']
    ranking_list_before['live_room'].iloc[i] = text

In [16]:
ranking_list_before = ranking_list_before.drop(['tag', 'description', 'title', 'name', 'sign'], axis=1)

In [17]:
ranking_list_before.to_csv(current_path + '/data_ranking_list_before.csv')

In [ ]:
# Ranking List After

In [11]:
current_path= os.getcwd()
ranking_list_after = pd.read_csv(current_path + '/ranking_list_after.csv', index_col='pubdate')

In [12]:
ranking_list_after.head(10)

,bvid,tag,duration,review,is_union_video,play,description,video_review,favorites,title,video_zone,name,sex,sign,level,birthday,official,vip,live_room
pubdate,,,,,,,,,,,,,,,,,,,
2020-01-01 00:47:32,BV1JJ411b7gX,"华语现场,腾格尔",294,505,0,757144,芒果台,3520.0,3750.0,腾格尔 《野狼disco》,29,ncbdwss,保密,NaN,5,NaN,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 1, 'status': 1, 'due_date': 162048960...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 02:29:23,BV1nJ411b7dn,"萨顶顶,LIVE现场,左手指月,冰雪盛典",150,636,0,526225,beijingzhibo.com,1126.0,2341.0,【萨顶顶】《左手指月》2020北京卫视环球跨年冰雪盛典超清版,29,一星丛一,男,王者乘时，圣人乘易，我顶乘仙,5,01-06,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 0, 'status': 0, 'due_date': 0, 'vip_p...","{'roomStatus': 0, 'liveStatus': 0, 'url': '', ..."
2020-01-01 02:55:11,BV1pJ411b78h,"冯提莫,潘玮柏,现场,不得不爱,LIVE",175,1305,0,616850,weibo,2124.0,2112.0,【潘玮柏&amp;冯提莫】《不得不爱》提莫甜美开唱，2019-2020江苏卫视跨年演唱会cut,29,我没捞着饭啊,保密,NaN,6,12-19,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 2, 'status': 1, 'due_date': 165746880...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 10:00:57,BV1nJ411h7xi,"全民音乐UP主,音乐,前男友,前任,兄弟,爱情,说唱,原创音乐,搞笑,动感视频",35,796,0,801600,NaN,279.0,5160.0,兄弟姐妹们！这到底咋劝啊！啊？,130,O_WEN欧文,保密,工作联系：Tel：17600710778/V：owzl9898\n微博：OW_欧文\n网易云...,6,NaN,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 0, 'status': 0, 'due_date': 0, 'vip_p...","{'roomStatus': 0, 'liveStatus': 0, 'url': '', ..."
2020-01-01 13:05:57,BV1bJ411b7cB,"玩具,弹珠,童年,DIY,速度,竞赛,野生技术协会,弹珠联盟,比赛,趣味科普人文",1219,578,0,576129,转载：https://www.youtube.com/watch?v=VQebp-AA4PY...,3741.0,8622.0,2019弹珠比赛10大高光时刻,122,盛尚sheng,保密,深夜奇思话题22：30开始欢迎大家参与讨论。,6,NaN,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 1, 'status': 1, 'due_date': 162169920...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 13:12:39,BV1zJ411b7SM,"华语现场,国语现场,音乐现场,现场,音乐,LIVE,开口跪,流行,跨年晚会,江苏卫视",238,787,0,788690,https://v.youku.com/v_show/id_XNDQ5MDIwMjA2MA=...,2370.0,10490.0,2020江苏卫视跨年晚会，王力宏、谭维维再现《缘分一道桥》超强现场，完美诠释中国风,29,zhenggot,保密,NaN,5,03-22,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 0, 'status': 0, 'due_date': 0, 'vip_p...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 13:14:11,BV1zJ411b7Kp,"华语现场,跨年演唱会,江苏卫视,GEM,陈伟霆,LIVE",244,431,0,798792,江苏卫视跨年演唱会\n江苏卫视2020跨年演唱会 宝石GEM、陈伟霆《野狼disco》蹦迪现场,1387.0,3814.0,江苏卫视2020跨年演唱会 宝石GEM、陈伟霆《野狼disco》蹦迪现场,29,六楼小乖乖,保密,NaN,4,NaN,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 0, 'status': 0, 'due_date': 0, 'vip_p...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 19:00:00,BV1pJ411h7s8,"但是还有书籍,胡歌,读书,文学,b站读书,And Yet The Books,自制,罗颖鸾,...",1746,688,0,795393,思想无形，幸好有了语言，有了文字；记忆是不可靠的东西，幸好有了书籍；时间太快，人们太忙，选择...,5946.0,113.0,【纪录片】但是还有书籍 第4集 设计师的纸上王国,37,哔哩哔哩纪录片,男,莫得感情偶尔咕咕的纪录菌,6,01-01,"{'role': 3, 'title': '哔哩哔哩纪录片官方账号', 'desc': ''...","{'type': 2, 'status': 1, 'due_date': 194664960...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."
2020-01-01 19:22:07,BV1mJ411b7Hi,"乐评盘点,精选歌单,BGM,开口跪,电音,欧美音乐",233,1595,0,5080778,"Blame - Calvin Harris,John Newman\n灭霸混剪：av8115...",5161.0,65083.0,终于找到这首一直“哒哒哒”的歌了，旋律太洗脑了,130,音乐斯图秀,男,网易歌单/微博同名\n交流群：258363472,6,02-27,"{'role': 0, 'title': '', 'desc': '', 'type': -1}","{'type': 2, 'status': 1, 'due_date': 162662400...","{'roomStatus': 1, 'liveStatus': 0, 'url': 'htt..."


In [13]:
ranking_list_after['sex'] = ranking_list_after['sex'].map({'男': 0, '女': 1, '保密': 2})

row_num = ranking_list_after.shape[0]

for i in range(0, row_num):
    content = ranking_list_after['official'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['role']
    ranking_list_after['official'].iloc[i] = text
    
for i in range(0, row_num):
    content = ranking_list_after['vip'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['type']
    ranking_list_after['vip'].iloc[i] = text  
    
for i in range(0, row_num):
    content = ranking_list_after['live_room'].iloc[i]
    temp = ast.literal_eval(content)
    text = temp['roomStatus']
    ranking_list_after['live_room'].iloc[i] = text    

/home/xyk/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [18]:
ranking_list_after = ranking_list_after.drop(['tag', 'description', 'title', 'name', 'sign'], axis=1)

In [19]:
ranking_list_after.to_csv(current_path + '/data_ranking_list_after.csv')

# Pinyin Convert

In [4]:
import os, fnmatch
import pandas as pd
from pypinyin import pinyin, lazy_pinyin, Style

In [7]:
current_path= os.getcwd()
temp = pd.read_csv(current_path + '/data_danmu/data_danmu_after_32.csv')

In [8]:
temp

,bvid,bgm,op,yi ge,yi yang,yi ji,san xuan,bu hui,bu kui,bu shi,...,zheng ming ti,hai shi,hai you,zhe ge,zhe me,zhe shi,zhe yang,zhe ji,na ge,gao mu
0,BV12b41137sM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BV12b41137sM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BV12b41137sM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BV12b41137sM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BV12b41137sM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743972,BV1wi4y1b7rM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4743973,BV1wi4y1b7rM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4743974,BV1wi4y1b7rM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4743975,BV1wi4y1b7rM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
header = temp.columns.values
header_num = len(header)
new_header = []
str = ' '

for i in range(0, header_num):
    text = str.join(lazy_pinyin(header[i]))
    new_header.append(text)
    
#new_header[0] = 'bvid'

In [5]:
temp.to_csv(current_path + '/data_danmu/data_danmu_after_32.csv', header=new_header, index=False)